In [ ]:
import pandas as pd
from pathlib import Path


In [ ]:

FILE_PATH = Path("/Users/nataliapodziolkina/Documents/final_project/Zomato Dataset.csv")
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

RATING_BINS = [0, 3, 4, 5]
RATING_LABELS = ['low (0-3)', 'medium (3-4)', 'high (4-5)']

df = pd.read_csv(FILE_PATH)

df.head()



In [ ]:
# Date cleaning 

df["Order_Date"] = (
    df["Order_Date"].astype(str).str.strip().str.replace(r"[-./]", "-", regex=True)
)
df["Order_Date"] = pd.to_datetime(df["Order_Date"], errors="coerce", dayfirst=True)



In [ ]:
# Hours
df["Time_Orderd"] = pd.to_datetime(df["Time_Orderd"], format="%H:%M", errors="coerce").dt.time
df["Time_Order_picked"] = pd.to_datetime(df["Time_Order_picked"], format="%H:%M", errors="coerce").dt.time


In [ ]:
# Drop missing daata
df_clean = df.dropna()

# delete duplicates 
df_clean = df_clean.drop_duplicates(subset=["ID"], keep="first")


In [ ]:
OUTPUT_CSV = OUTPUT_DIR / "zomato_clean_dropna.csv"
df_clean.to_csv(OUTPUT_CSV, index=False)

print("✅ Cleaned dataset saved:", OUTPUT_CSV)
print("Кількість рядків після очищення:", len(df_clean))



In [ ]:
# Orders per City

orders_city = df_clean.groupby("City").agg(
    total_orders=("ID", "count"),
    avg_time=("Time_taken (min)", "mean")
).round(2).reset_index()

orders_city.to_csv(OUTPUT_DIR / "orders_city.csv", index=False)


In [ ]:
#  Orders per Traffic
orders_traffic = df_clean.groupby("Road_traffic_density").agg(
    total_orders=("ID", "count"),
    avg_time=("Time_taken (min)", "mean")
).round(2).reset_index()

orders_traffic.to_csv(OUTPUT_DIR / "orders_traffic.csv", index=False)




In [ ]:
#  Orders per Weather
orders_weather = df_clean.groupby("Weather_conditions").agg(
    total_orders=("ID", "count"),
    avg_time=("Time_taken (min)", "mean")
).round(2).reset_index()

orders_weather.to_csv(OUTPUT_DIR / "orders_weather.csv", index=False)


In [ ]:
#  Festival impact
orders_festival = df_clean.groupby("Festival").agg(
    total_orders=("ID", "count"),
    avg_time=("Time_taken (min)", "mean")
).round(2).reset_index()

orders_festival.to_csv(OUTPUT_DIR / "orders_festival.csv", index=False)



In [ ]:
#  Type of the order
orders_type = df_clean.groupby("Type_of_order").agg(
    total_orders=("ID", "count"),
    avg_time=("Time_taken (min)", "mean")
).round(2).reset_index()

orders_type.round(2).to_csv(OUTPUT_DIR / "orders_type.csv", index=False)




In [ ]:

#  Courier productivity
courier_load = df_clean.groupby("Delivery_person_ID").agg(
    total_orders=("ID", "count"),
    avg_rating=("Delivery_person_Ratings", "mean")
).round(2).reset_index()

courier_load.to_csv(OUTPUT_DIR / "courier_load.csv", index=False)




In [ ]:
# Trend by date
df_clean["Order_Date"] = pd.to_datetime(df_clean["Order_Date"])
orders_trend = df_clean.groupby("Order_Date").agg(
    total_orders=("ID", "count"),
    avg_time=("Time_taken (min)", "mean")
).round(2).reset_index()


In [ ]:
# Rating vs delivery time 
ratings_time = df_clean.groupby(
    pd.cut(df_clean["Delivery_person_Ratings"], bins=RATING_BINS, labels=RATING_LABELS),
    observed=True,
).agg(
    avg_time=("Time_taken (min)", "mean"),
    total_orders=("ID", "count")
).round(2).reset_index()

ratings_time.to_csv(OUTPUT_DIR / "ratings_time.csv", index=False)

